# Visualization Example using Pycortex 

In [1]:
import cortex
import numpy as np
import sys
import h5py
import os
import tables

# sys.path.append('/path/to/config.py')
sys.path.append('/home/projects/dialogue/code/')
from config import DATA_DIR, RESULT_DIR, SUBJECTS_ALL, TOTAL_SESS, LAYERS, CL, NVOXELS
from code.utils_pycortex import (
    get_volume_neg2nan_ALLSUBJ_base, get_volume_context_neg2nan_ALLSUBJ,
    get_volume_cross_context_neg2nan_ALLSUBJ, get_volume2D_bestvp_neg2nan_ALLSUBJ,
    get_volume2D_context_weightcorr_bothsignificant, get_volume_weightcorr_VP_mask,
    get_volume2D_context_all_weightcorr_significant_unified_diff
    )


               Random Search + Gradient Descent 


## files

In [2]:
### Head Motion model results
head_motion_model_file=f"{RESULT_DIR}ALLSUBJ_sig_brainscore_step1_headmotion.hdf"

### Random embedding model results
random_model_file=f"{RESULT_DIR}ALLSUBJ_sig_brainscore_step2_randnormal.hdf"

### Separate Linguistic model results
separate_model_file=f"{RESULT_DIR}ALLSUBJ_sig_brainscore_step3_gpt_intramodal.hdf"

### Separate Linguistic model cross-modality prediction results
cross_modality_file=f"{RESULT_DIR}ALLSUBJ_sig_brainscore_step3_gpt_crossmodal.hdf"

### Unified Linguistic model results
unified_model_file=f"{RESULT_DIR}ALLSUBJ_sig_brainscore_step3_gpt_unified.hdf"

### Best variance partition results
bestbp_file=f"{RESULT_DIR}ALLSUBJ_best_varpart.hdf"

### Weight correlation in the Separate Linguistic model weights
wc_sep_file=f"{RESULT_DIR}ALLSUBJ_weightcorrs_gpt_separate.hdf"

### Weight correlation similarity between the Separate Linguistic model weights with Unified Linguistic model weights
wc_unif_sep_file=f"{RESULT_DIR}ALLSUBJ_weightcorrs_gpt_unified.hdf"



# Prediction accuracy (brain score)

## Head Motion model/Random embedding model (Supplementary Figs.3 and 4)

In [3]:
cmap='inferno'
VMAX=0.3

### files
# head_motion_model_file=f"{RESULT_DIR}ALLSUBJ_sig_brainscore_step1_headmotion.hdf"
# random_model_file=f"{RESULT_DIR}ALLSUBJ_sig_brainscore_step2_randnormal.hdf"

volumes={}
for subject_idx in range(8):
    subject = SUBJECTS_ALL[subject_idx]

    ### Head Motion model
    fig_name=f"{subject}_headmotion"
    if os.path.exists(head_motion_model_file):
        volumes[fig_name]=get_volume_neg2nan_ALLSUBJ_base(subject_idx, subject, head_motion_model_file, cmap=cmap, vmax=VMAX)
    else:
        print("Not available {model_file}")

    ### Random embedding model
    fig_name=f"{subject}_random"
    if os.path.exists(random_model_file):
        volumes[fig_name]=get_volume_neg2nan_ALLSUBJ_base(subject_idx, subject, random_model_file, cmap=cmap, vmax=VMAX)
    else:
        print("Not available {model_file}")
    
im =cortex.webshow(volumes)


Started server on port 29275
Stopping server


## Separte and Unified Linguistic model (Fig 2a, Supplementary Figs 5-7)

In [4]:
cmap='hot'

THR=0
VMIN=0
VMAX=0.3

################################################################################################
volumes={}
for subject_idx in range(8):
    subject = SUBJECTS_ALL[subject_idx]
    for layer_idx, layer in enumerate(LAYERS):
        if layer == 18:
            for cl_idx, cl in enumerate(CL):
                if cl in [1, 8, 32]:
                    ### Separate Linguistic model 
                    # separate_model_file=f"{RESULT_DIR}ALLSUBJ_sig_brainscore_step3_gpt_intramodal.hdf"
                    # cross_modality_file=f"{RESULT_DIR}ALLSUBJ_sig_brainscore_step3_gpt_crossmodal.hdf"
                    # unified_model_file=f"{RESULT_DIR}ALLSUBJ_sig_brainscore_step3_gpt_unified.hdf"
                    test_idx=1
                    if os.path.exists(separate_model_file):
                        fig_name=f"{subject}_Separete_L-{layer}_CL-{cl}"
                        volumes[fig_name]=get_volume_context_neg2nan_ALLSUBJ(subject_idx, subject, layer_idx, cl_idx, test_idx, separate_model_file, cmap=cmap, vmin=VMIN, vmax=VMAX, thr=THR)
                    else:
                        print(f"Not available {separate_model_file}")

                    ### Separate Linguistic model cross-modality
                    if os.path.exists(cross_modality_file):
                        fig_name=f"{subject}_Cross_L-{layer}_CL-{cl}"
                        volumes[fig_name]=get_volume_cross_context_neg2nan_ALLSUBJ(subject_idx, subject, layer_idx, cl_idx, cross_modality_file, separate_model_file, cmap=cmap, vmin=VMIN, vmax=VMAX, thr=THR)
                    else:
                        print(f"Not available {cross_modality_file}")

                    ### Unified Linguistic model
                    if os.path.exists(unified_model_file):
                        fig_name=f"{subject}_Unified_L-{layer}_CL-{cl}"
                        volumes[fig_name]=get_volume_context_neg2nan_ALLSUBJ(subject_idx, subject, layer_idx, cl_idx, test_idx, unified_model_file, cmap=cmap, vmin=VMIN, vmax=VMAX, thr=THR)
                    else:
                        print(f"Not available {unified_model_file}")
                    

im =cortex.webshow(volumes)


Started server on port 38127
Stopping server


# Variance partitioning

## Best variance partitioning (Fig 4c, Supplementary Fig 15)

In [5]:
cmap="BROYG_2D"

# bestbp_file=f"{RESULT_DIR}ALLSUBJ_best_varpart.hdf"
# separate_model_file=f"{RESULT_DIR}ALLSUBJ_sig_brainscore_step3_gpt_intramodal.hdf"
THR=0.05
VMAX=0.2

volumes={}
for subject_idx in range(8):
    subject = SUBJECTS_ALL[subject_idx]    
    for layer_idx, layer in enumerate(LAYERS):
        if layer == 18:
            for cl_idx, cl in enumerate(CL):
                if cl in [1, 8, 32]:
                    fig_name=f"{subject}_l-{layer}_s-{cl}"
                    volumes[fig_name]=get_volume2D_bestvp_neg2nan_ALLSUBJ(subject_idx, subject, layer_idx, cl_idx, bestbp_file, separate_model_file, vmin=THR, vmax=VMAX, thr=THR, cmap=cmap)

im =cortex.webshow(volumes)


Started server on port 29645
Stopping server


# Weight correlation

## WC in cross-modal voxels, bimodal voxels, and differences (Fig 2c/SFig 13, 3d/SFig 14, 4e/SFig 16)

In [6]:
### parameters
TARGET_LAYER=18

##########################################################################################
volumes={}
for subject_idx in range(8):
    subject = SUBJECTS_ALL[subject_idx]
    for cl_idx, cl in enumerate(CL):
        if cl in [1, 8, 32]:
            for layer_idx, layer in enumerate(LAYERS):
                if layer == TARGET_LAYER:
                    ### cross-modal voxels
                    ## Fig. 2c WC in cross-modal voxels (Supplementary Fig 13)
                    fig_name=f"{subject}_l-{layer}_{cl}_crossmodal_voxels"
                    cmap="BuWtRd_alpha"
                    WC_MAX=0.4
                    SCORE_MIN=0.05
                    SCORE_MAX=0.2
                    volumes[fig_name] = get_volume2D_context_weightcorr_bothsignificant(subject, subject_idx, layer_idx, cl_idx, wc_sep_file, separate_model_file, cross_modality_file, 
                                                                                vmin=-WC_MAX, vmax=WC_MAX, score_min=SCORE_MIN, score_max=SCORE_MAX, cmap=cmap)

                    ### bimodal voxels
                    modality_idx=3
                    ## Fig. 4e WC in bimodal voxels (Supplementary Fig 16)
                    fig_name=f"{subject}_l-{layer}_{cl}_bimodal_voxels"
                    volumes[fig_name] = get_volume_weightcorr_VP_mask(subject, subject_idx, layer_idx, cl_idx, modality_idx, wc_sep_file, bestbp_file, separate_model_file, 
                                                                                vmin=-WC_MAX, vmax=WC_MAX, cmap=cmap)

                    
                    ### wc differences 
                    ## Fig 3d WC difference between Unified model weights and Separate model weights (comp - prod) in unified scores (Supplementary Fig 14)
                    fig_name=f"{subject}_l-{layer}_{cl}_diff"
                    cmap="BuBkRd_alpha_2D"
                    WC_MAX=0.3
                    SCORE_MIN=0.05
                    SCORE_MAX=0.3
                    volumes[fig_name] = get_volume2D_context_all_weightcorr_significant_unified_diff(subject, subject_idx, layer_idx, cl_idx, wc_unif_sep_file, separate_model_file, unified_model_file, 
                                                                                vmin=-WC_MAX, vmax=WC_MAX, score_min=SCORE_MIN, score_max=SCORE_MAX, cmap=cmap)

im =cortex.webshow(volumes)


# of prod, comp, bimodal voxels: (23400, 18908, 24659)
# of prod, comp, bimodal voxels: (25728, 20887, 18732)
# of prod, comp, bimodal voxels: (19385, 17806, 28672)
# of prod, comp, bimodal voxels: (25203, 20087, 21282)
# of prod, comp, bimodal voxels: (28382, 14295, 23177)
# of prod, comp, bimodal voxels: (18436, 21497, 23231)
# of prod, comp, bimodal voxels: (29865, 14066, 18888)
# of prod, comp, bimodal voxels: (16684, 18445, 26945)
# of prod, comp, bimodal voxels: (17616, 20106, 23528)
# of prod, comp, bimodal voxels: (26633, 15996, 17929)
# of prod, comp, bimodal voxels: (14212, 23827, 21917)
# of prod, comp, bimodal voxels: (10455, 26676, 21216)
# of prod, comp, bimodal voxels: (21427, 20442, 22779)
# of prod, comp, bimodal voxels: (14379, 29095, 20715)
# of prod, comp, bimodal voxels: (15351, 27679, 20109)
# of prod, comp, bimodal voxels: (22442, 20590, 24644)
# of prod, comp, bimodal voxels: (15816, 28302, 22404)
# of prod, comp, bimodal voxels: (11033, 32782, 21005)
# of prod,